In [ ]:
import os
# Temporarily unset MLRUN_DBPATH to allow the import to succeed if the bug is in its processing
original_dbpath = os.environ.pop("MLRUN_DBPATH", None) 

import mlrun
print(f"MLRun Version: {mlrun.get_version()}")

# Restore it if it was there, though it's causing issues
if original_dbpath:
    os.environ["MLRUN_DBPATH"] = original_dbpath

In [1]:
import os
import logging
import shutil

# --- 0. Logging ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT) 
logger = logging.getLogger("MLRun141NewProj")
mlrun_lib_logger = logging.getLogger("mlrun")
mlrun_lib_logger.setLevel(logging.WARNING)

logger.info("--- MLRun 1.4.1 new_project Attempt ---")

# --- Paths ---
NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "mlrun141np_artifacts")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "mlrun141np.db") # New DB name
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}"
PROJECT_NAME = "mlrun141np-project" # New project name
FUNCTION_FILE_NAME = "mlrun141np_func.py"
DOCKER_IMAGE = "mlrun/mlrun"

# Clean slate
logger.info(f"Cleaning up: {DB_FILE_PATH}, project.yaml for {PROJECT_NAME}, {ARTIFACT_PATH}")
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
project_yaml_path = os.path.join(NOTEBOOK_DIR, "project.yaml")
if os.path.exists(project_yaml_path): os.remove(project_yaml_path)
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)
logger.info("Cleanup done.")

# --- 1. Set Env Vars THEN Import MLRun ---
logger.info(f"Setting OS Env MLRUN_DBPATH = {DB_CONNECTION_STRING}")
os.environ["MLRUN_DBPATH"] = DB_CONNECTION_STRING
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH
for var in ["MLRUN_HTTPDB__URL", "MLRUN_API_PROXY__URL", "MLRUN_REMOTE_HOST", "MLRUN_EXEC_CONFIG"]:
    if var in os.environ: del os.environ[var]

import mlrun
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should be 1.4.1

# --- 2. Verify DB setup (should be correct with 1.4.1) ---
logger.info(f"VERIFY: mlrun.mlconf.dbpath = {mlrun.mlconf.dbpath}")
if mlrun.mlconf.dbpath != DB_CONNECTION_STRING:
    logger.error("CRITICAL MISMATCH! mlrun.mlconf.dbpath not picked up. Halting.")
    raise RuntimeError("mlconf.dbpath mismatch")
logger.info("mlrun.mlconf.dbpath matches env var.")

try:
    db = mlrun.get_run_db() 
    logger.info(f"mlrun.get_run_db() returned: {type(db)}")
    if "SQLDB" not in str(type(db)): # 1.4.1 uses SQLDB for sqlite
         logger.error(f"DB type is {type(db)}, NOT SQLDB. Halting.")
         raise RuntimeError("DB is not SQLDB")
    logger.info("DB type is SQLDB (Correct).")
except Exception as e:
    logger.error(f"Error getting DB: {e}", exc_info=True)
    raise

# --- 3. Use new_project() instead of get_or_create_project() ---
logger.info(f"DB file '{DB_FILE_PATH}' exists before new_project: {os.path.exists(DB_FILE_PATH)}")
try:
    # Create the project object in memory first, don't save immediately
    logger.info(f"Attempting mlrun.new_project(name='{PROJECT_NAME}', context='./', save=False)...")
    project = mlrun.new_project(name=PROJECT_NAME, context="./", user_project=False, save=False) 
    logger.info(f"Project object '{project.name}' created in memory. Attempting project.save()...")
    
    # Now try saving it explicitly. This is where the DB interaction happens.
    project.save()
    logger.info("project.save() executed.")

except RecursionError:
    logger.error("RECURSION ERROR HIT even with new_project + save. The save() method likely still triggers the bad get_project call internally.")
    raise
except AttributeError as ae:
     logger.error(f"AttributeError during new_project/save: {ae}", exc_info=True)
     logger.error("This might be the 'NoneType' has no attribute '...' error if the DB singleton issue persists implicitly.")
     raise
except Exception as e_proj:
    logger.error(f"Error during new_project or project.save(): {e_proj}", exc_info=True)
    raise

# --- 4. Check DB File Creation ---
if not os.path.exists(DB_FILE_PATH):
    logger.error(f"FAILURE: DB file '{DB_FILE_PATH}' NOT FOUND after project.save().")
    raise RuntimeError("DB file not created by project.save() using new_project approach.")
else:
    logger.info(f"SUCCESS: DB file '{DB_FILE_PATH}' EXISTS after project.save().")
    
    # --- 5. Function definition and run (IF DB was created successfully) ---
    func_code = "def handler(context): context.log_result('test_141np', 'ok_141np')"
    with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)

    fn = project.set_function(FUNCTION_FILE_NAME, name="mlrun141np-job", kind="job", image=DOCKER_IMAGE, handler="handler")
    
    logger.info("Attempting to run function with local=False...")
    try:
        run_obj = project.run_function(fn, local=False, watch=True)
        logger.info(f"Run state: {run_obj.status.state if run_obj else 'N/A'}")
        if run_obj and run_obj.status.state == "error":
             logger.error(f"Run failed: {run_obj.status.error}")
    except Exception as e:
        logger.error(f"Error during run_function: {e}", exc_info=True)

logger.info("--- MLRun 1.4.1 new_project Attempt End ---")

2025-05-07 16:56:35,190 INFO [MLRun141NewProj] 1824500501.py:12 - --- MLRun 1.4.1 new_project Attempt ---
2025-05-07 16:56:35,191 INFO [MLRun141NewProj] 1824500501.py:24 - Cleaning up: /home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/mlrun141np.db, project.yaml for mlrun141np-project, /home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/mlrun141np_artifacts
2025-05-07 16:56:35,192 INFO [MLRun141NewProj] 1824500501.py:30 - Cleanup done.
2025-05-07 16:56:35,192 INFO [MLRun141NewProj] 1824500501.py:33 - Setting OS Env MLRUN_DBPATH = sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/mlrun141np.db
2025-05-07 16:56:36,335 INFO [MLRun141NewProj] 1824500501.py:40 - MLRun Imported. Version: 1.4.1
2025-05-07 16:56:36,336 INFO [MLRun141NewProj] 1824500501.py:43 - VERIFY: mlrun.mlconf.dbpath = sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/mlrun141np.db
2025-05-07 16:56:36,336 INFO [MLRun141NewProj] 1824500501.py:47 - mlrun.mlconf.db

AttributeError: 'NoneType' object has no attribute 'store_project'

In [ ]:
import os
import logging
import shutil

# --- 0. Logging ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("MonkeyPatchAttempt")
mlrun_lib_logger = logging.getLogger("mlrun")
mlrun_lib_logger.setLevel(logging.WARNING)

logger.info("--- Monkey Patch Attempt for MLRun DB ---")

# --- Paths ---
NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "mp_artifacts")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "mp_mlrun.db")
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}" # The actual SQLite path
PROJECT_NAME = "mp-project"

# Clean slate
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
if os.path.exists(os.path.join(NOTEBOOK_DIR, "project.yaml")): os.remove(os.path.join(NOTEBOOK_DIR, "project.yaml"))
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)

# --- 1. Import MLRun and related components ---
# Set MLRUN_DBPATH to an EMPTY string to hopefully make it default to NopDB initially
# without triggering the error during import.
os.environ["MLRUN_DBPATH"] = ""
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH
for var in ["MLRUN_HTTPDB__URL", "MLRUN_API_PROXY__URL", "MLRUN_REMOTE_HOST", "MLRUN_EXEC_CONFIG"]:
    if var in os.environ: del os.environ[var]

import mlrun
from mlrun.db.sqldb import SQLDB # Try to import the specific SQLDB class
from mlrun.db.filedb import FileDB # For older versions, FileDB might be the one for SQLite
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should be 1.5.0 now

# --- 2. MONKEY PATCHING / DIRECT DB INSTANCE INJECTION ---
logger.info(f"Initial mlrun.mlconf.dbpath: '{mlrun.mlconf.dbpath}'")
logger.info(f"Initial resolved DB type by mlrun.get_run_db(): {type(mlrun.get_run_db())}")

logger.info(f"Attempting to manually create and inject SQLDB/FileDB instance for: {DB_CONNECTION_STRING}")

try:
    # Option A: Try SQLDB first (more common in later versions for SQLite)
    try:
        logger.info("Trying to instantiate SQLDB directly...")
        manual_db_instance = SQLDB(dsn=DB_CONNECTION_STRING)
        manual_db_instance.connect() # Explicitly connect
        logger.info(f"Manually created SQLDB instance: {manual_db_instance}")
    except Exception as e_sqldb:
        logger.warning(f"Failed to create SQLDB directly: {e_sqldb}. Trying FileDB...")
        # Option B: Try FileDB (might be used for SQLite in older versions like 1.5.0)
        manual_db_instance = FileDB(dirpath=NOTEBOOK_DIR, dsn=DB_CONNECTION_STRING) # FileDB might use dirpath
        manual_db_instance.connect()
        logger.info(f"Manually created FileDB instance: {manual_db_instance}")

    # This is the dangerous part: directly replacing the internal DB object holder.
    # The exact internal variable might differ slightly between versions.
    # Common places are mlrun.db._run_db or an attribute in mlrun.db.factory.run_db_factory
    if hasattr(mlrun.db, "_run_db"):
        mlrun.db._run_db = manual_db_instance
        logger.info(f"Monkey-patched mlrun.db._run_db with: {type(manual_db_instance)}")
    elif hasattr(mlrun.db.factory, "run_db_factory") and hasattr(mlrun.db.factory.run_db_factory, "_run_db"):
        mlrun.db.factory.run_db_factory._run_db = manual_db_instance
        logger.info(f"Monkey-patched mlrun.db.factory.run_db_factory._run_db with: {type(manual_db_instance)}")
    else:
        logger.error("Could not find the internal _run_db variable to monkey-patch.")
        raise RuntimeError("Failed to find internal DB instance holder for monkey-patching.")

    # After monkey-patching, verify what get_run_db() returns
    # It should now return our manually injected instance.
    current_db_after_patch = mlrun.get_run_db(force_reconnect=False) # Don't force reconnect, use current
    logger.info(f"DB type via get_run_db() after patch: {type(current_db_after_patch)}")
    if not isinstance(current_db_after_patch, (SQLDB, FileDB)):
        logger.error("Monkey-patching failed to make get_run_db() return the correct type!")
        raise RuntimeError("Patch failed.")
    
    # Set mlconf.dbpath to the *actual* string for other parts of MLRun that might read it,
    # *hoping* it doesn't trigger the buggy setter if the DB instance is already set.
    # This is risky. If this line *still* triggers the ValueError, the setter is too aggressive.
    logger.info("Attempting to set mlrun.mlconf.dbpath to actual string AFTER monkey-patching DB instance...")
    mlrun.mlconf.dbpath = DB_CONNECTION_STRING 
    logger.info(f"mlrun.mlconf.dbpath is now: {mlrun.mlconf.dbpath}")


except Exception as e:
    logger.error(f"Error during monkey-patching or DB setup: {e}", exc_info=True)
    raise # Stop if this desperate measure fails

# --- 3. Project and DB File Creation (Hoping the patched DB works) ---
logger.info(f"DB file '{DB_FILE_PATH}' exists before project ops: {os.path.exists(DB_FILE_PATH)}")
project = mlrun.get_or_create_project(name=PROJECT_NAME, context="./", user_project=False)
try:
    project.save() # This will use the (hopefully) patched DB object
    logger.info("project.save() executed.")
except Exception as e_save:
    logger.error(f"Error during project.save() with patched DB: {e_save}", exc_info=True)

if not os.path.exists(DB_FILE_PATH):
    logger.error(f"FAILURE: DB file '{DB_FILE_PATH}' NOT created after project.save() with patched DB.")
    raise RuntimeError("DB file not created even with monkey-patch.")
logger.info(f"SUCCESS: DB file '{DB_FILE_PATH}' EXISTS after project.save() with patched DB.")

# --- 4. Function Run (IF DB was created) ---
# (Minimal function definition and run)
FUNCTION_FILE_NAME = "mp_func.py"
DOCKER_IMAGE = "mlrun/mlrun"
func_code = "def handler(context): context.log_result('test_mp', 'ok_mp')"
with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)
fn = project.set_function(FUNCTION_FILE_NAME, name="mp-job", kind="job", image=DOCKER_IMAGE, handler="handler")
logger.info("Attempting to run function with local=False...")
try:
    run_obj = project.run_function(fn, local=False, watch=True)
    logger.info(f"Run state: {run_obj.status.state if run_obj else 'N/A'}")
    if run_obj and run_obj.status.state == "error":
        logger.error(f"Run failed: {run_obj.status.error}")
except Exception as e:
    logger.error(f"Error during run_function with patched DB: {e}", exc_info=True)

logger.info("--- Monkey Patch Attempt End ---")

In [1]:
import os
import logging
import shutil

# --- 0. Setup ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("StableVersionTest")

NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "stable_artifacts")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "stable_mlrun.db")
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}"
PROJECT_NAME = "stable-local-project"
FUNCTION_FILE_NAME = "stable_func.py"
DOCKER_IMAGE = "mlrun/mlrun" # Or a version-compatible image if needed

# Clean slate
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
if os.path.exists(os.path.join(NOTEBOOK_DIR, "project.yaml")): os.remove(os.path.join(NOTEBOOK_DIR, "project.yaml"))
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)

# --- 1. Set Env Vars THEN Import MLRun ---
logger.info(f"Setting OS Env MLRUN_DBPATH = {DB_CONNECTION_STRING}")
os.environ["MLRUN_DBPATH"] = DB_CONNECTION_STRING
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH
for var in ["MLRUN_HTTPDB__URL", "MLRUN_API_PROXY__URL", "MLRUN_REMOTE_HOST", "MLRUN_EXEC_CONFIG"]:
    if var in os.environ: del os.environ[var]

import mlrun
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should now be e.g., 1.7.0

logger.info(f"VERIFY: mlrun.mlconf.dbpath = {mlrun.mlconf.dbpath}")
if mlrun.mlconf.dbpath != DB_CONNECTION_STRING:
    logger.error("CRITICAL MISMATCH with stable version!")
else:
    logger.info("SUCCESS: mlrun.mlconf.dbpath matches env var with stable version.")

# Check the DB type MLRun resolved to
try:
    db = mlrun.get_run_db()
    logger.info(f"mlrun.get_run_db() returned: {type(db)}")
    if "HTTPRunDB" in str(type(db)):
        logger.error("STABLE VERSION STILL RESOLVED TO HTTPRunDB! This is unexpected.")
    elif "SQLDB" in str(type(db)) or "FileDB" in str(type(db)): # FileDB for older versions
        logger.info("SUCCESS: DB type is SQLDB/FileDB, correct for SQLite.")
    else:
        logger.warning(f"DB type is {type(db)}, was expecting SQLDB/FileDB.")
except Exception as e:
    logger.error(f"Error getting DB with stable version: {e}", exc_info=True)
    raise

# --- 2. Project and Function ---
project = mlrun.get_or_create_project(name=PROJECT_NAME, context="./", user_project=False)
project.save()
if not os.path.exists(DB_FILE_PATH):
    logger.error("DB file NOT created by project.save() with stable version.")
    raise RuntimeError("DB file not created with stable version")
logger.info("DB file created by project.save() with stable version.")

func_code = "def handler(context): context.log_result('test_stable', 'ok_stable')"
with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)

fn = project.set_function(FUNCTION_FILE_NAME, name="stable-job", kind="job", image=DOCKER_IMAGE, handler="handler")

logger.info("Attempting to run function with local=False with stable version...")
try:
    run_obj = project.run_function(fn, local=False, watch=True)
    logger.info(f"Run state (stable): {run_obj.status.state if run_obj else 'N/A'}")
    if run_obj and run_obj.status.state == "error":
         logger.error(f"Run failed (stable): {run_obj.status.error}")
except Exception as e:
    logger.error(f"Error during run_function (stable): {e}", exc_info=True)

logger.info("--- Stable Version Test End ---")

2025-05-07 16:53:28,201 INFO [StableVersionTest] 3870799193.py:25 - Setting OS Env MLRUN_DBPATH = sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/stable_mlrun.db
2025-05-07 16:53:29,332 INFO [StableVersionTest] 3870799193.py:32 - MLRun Imported. Version: 1.4.1
2025-05-07 16:53:29,332 INFO [StableVersionTest] 3870799193.py:34 - VERIFY: mlrun.mlconf.dbpath = sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/stable_mlrun.db
2025-05-07 16:53:29,332 INFO [StableVersionTest] 3870799193.py:38 - SUCCESS: mlrun.mlconf.dbpath matches env var with stable version.
2025-05-07 16:53:29,333 INFO [StableVersionTest] 3870799193.py:43 - mlrun.get_run_db() returned: <class 'mlrun.db.sqldb.SQLDB'>
2025-05-07 16:53:29,333 INFO [StableVersionTest] 3870799193.py:47 - SUCCESS: DB type is SQLDB/FileDB, correct for SQLite.


AttributeError: 'NoneType' object has no attribute 'get_project'

In [1]:
import os
import logging
import shutil

# --- 0. Logging ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("UltraMinimalForce")
mlrun_lib_logger = logging.getLogger("mlrun")
mlrun_lib_logger.setLevel(logging.WARNING) # Keep MLRun quiet unless errors

logger.info("--- Ultra Minimal Force Mlconf Test ---")

# --- Paths ---
NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "um_artifacts")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "um_mlrun.db")
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}"
PROJECT_NAME = "um-project"

# Clean slate
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
if os.path.exists(os.path.join(NOTEBOOK_DIR, "project.yaml")): os.remove(os.path.join(NOTEBOOK_DIR, "project.yaml"))
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)

# --- 1. Import MLRun (Env vars for DBPATH are being ignored, so don't rely on them here for dbpath) ---
# We can still set artifact_path via env var as it's less problematic generally
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH 
# Explicitly ensure MLRUN_DBPATH env var is NOT set or is empty to avoid confusion if it *sometimes* works
if "MLRUN_DBPATH" in os.environ:
    del os.environ["MLRUN_DBPATH"]
logger.info(f"MLRUN_DBPATH env var before import: {os.environ.get('MLRUN_DBPATH')}")

import mlrun
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should be 1.7.0

# --- 2. CRITICAL: FORCE mlrun.mlconf IMMEDIATELY AFTER IMPORT ---
logger.info(f"MLRun's initial mlrun.mlconf.dbpath (after import): '{mlrun.mlconf.dbpath}'")

logger.info(f"FORCING mlrun.mlconf.dbpath to: '{DB_CONNECTION_STRING}'")
mlrun.mlconf.dbpath = DB_CONNECTION_STRING

# Also force artifact_path just in case, and ensure httpdb.url is empty
mlrun.mlconf.artifact_path = ARTIFACT_PATH 
if hasattr(mlrun.mlconf, "httpdb") and hasattr(mlrun.mlconf.httpdb, "url"):
    mlrun.mlconf.httpdb.url = ""

logger.info(f"VERIFY FORCED: mlrun.mlconf.dbpath = '{mlrun.mlconf.dbpath}'")
logger.info(f"VERIFY FORCED: mlrun.mlconf.artifact_path = '{mlrun.mlconf.artifact_path}'")

# --- 3. Get DB and Check Type (using force_reconnect) ---
try:
    db = mlrun.get_run_db(force_reconnect=True) 
    logger.info(f"mlrun.get_run_db(force_reconnect=True) returned type: {type(db)}")
    if "SQLDB" not in str(type(db)) and "FileDB" not in str(type(db)):
        logger.error(f"DB type is {type(db)}, NOT SQLDB/FileDB. This is the problem point!")
        raise RuntimeError(f"DB resolved to {type(db)} instead of SQLDB/FileDB after forcing mlconf.")
    logger.info("DB type is SQLDB/FileDB (Correct).")
except Exception as e:
    logger.error(f"Error from mlrun.get_run_db() even after forcing mlconf: {e}", exc_info=True)
    raise # Stop if we can't even get the right DB object

# --- 4. Project and DB File Creation ---
logger.info(f"DB file '{DB_FILE_PATH}' exists before project ops: {os.path.exists(DB_FILE_PATH)}")
project = mlrun.get_or_create_project(name=PROJECT_NAME, context="./", user_project=False)
try:
    project.save()
    logger.info("project.save() executed.")
except Exception as e_save:
    logger.error(f"Error during project.save(): {e_save}", exc_info=True)
    # Fall through to check file existence

if not os.path.exists(DB_FILE_PATH):
    logger.error(f"FAILURE: DB file '{DB_FILE_PATH}' NOT created after project.save().")
    raise RuntimeError("DB file not created by project.save() even after forcing mlconf.")
logger.info(f"SUCCESS: DB file '{DB_FILE_PATH}' EXISTS after project.save().")

# --- 5. Function Run (IF DB was created successfully) ---
# (Minimal function definition and run, similar to previous attempts)
FUNCTION_FILE_NAME = "um_func.py"
DOCKER_IMAGE = "mlrun/mlrun"
func_code = "def handler(context): context.log_result('test_um_force', 'ok_um_force')"
with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)

fn = project.set_function(FUNCTION_FILE_NAME, name="um-force-job", kind="job", image=DOCKER_IMAGE, handler="handler")
logger.info("Attempting to run function with local=False...")
try:
    run_obj = project.run_function(fn, local=False, watch=True)
    logger.info(f"Run state: {run_obj.status.state if run_obj else 'N/A'}")
    if run_obj and run_obj.status.state == "error":
            logger.error(f"Run failed: {run_obj.status.error}")
except Exception as e:
    logger.error(f"Error during run_function: {e}", exc_info=True)

logger.info("--- Ultra Minimal Force Mlconf Test End ---")

2025-05-07 16:50:32,955 INFO [UltraMinimalForce] 3594566908.py:12 - --- Ultra Minimal Force Mlconf Test ---
2025-05-07 16:50:32,956 INFO [UltraMinimalForce] 3594566908.py:33 - MLRUN_DBPATH env var before import: None
2025-05-07 16:50:34,071 INFO [UltraMinimalForce] 3594566908.py:36 - MLRun Imported. Version: 1.4.1
2025-05-07 16:50:34,072 INFO [UltraMinimalForce] 3594566908.py:39 - MLRun's initial mlrun.mlconf.dbpath (after import): ''
2025-05-07 16:50:34,072 INFO [UltraMinimalForce] 3594566908.py:41 - FORCING mlrun.mlconf.dbpath to: 'sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/um_mlrun.db'
2025-05-07 16:50:34,077 INFO [UltraMinimalForce] 3594566908.py:49 - VERIFY FORCED: mlrun.mlconf.dbpath = 'sqlite:////home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/um_mlrun.db'
2025-05-07 16:50:34,077 INFO [UltraMinimalForce] 3594566908.py:50 - VERIFY FORCED: mlrun.mlconf.artifact_path = '/home/johnny/swan/code/local-mrlrun/notebooks/01_basic_tests/um_artifacts

AttributeError: 'NoneType' object has no attribute 'get_project'

In [ ]:
import os
import logging
import shutil
import sqlite3 # Import for direct SQLite check
import mlrun # Keep this import after env vars are set

# --- 0. Setup ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("StableVersionDBTest")

NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "stable_artifacts_dbg")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "stable_mlrun_dbg.db") # New DB name for this test
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}"
PROJECT_NAME = "stable-local-project-dbg"
FUNCTION_FILE_NAME = "stable_func_dbg.py"
DOCKER_IMAGE = "mlrun/mlrun"

# Clean slate
logger.info(f"Attempting to clean up: {DB_FILE_PATH}, project.yaml, {ARTIFACT_PATH}")
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
project_yaml_path = os.path.join(NOTEBOOK_DIR, "project.yaml") # Assuming default name
if os.path.exists(project_yaml_path): os.remove(project_yaml_path)
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)
logger.info("Cleanup done.")

# --- 1. Set Env Vars THEN Import MLRun ---
logger.info(f"Setting OS Env MLRUN_DBPATH = {DB_CONNECTION_STRING}")
os.environ["MLRUN_DBPATH"] = DB_CONNECTION_STRING
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH
for var in ["MLRUN_HTTPDB__URL", "MLRUN_API_PROXY__URL", "MLRUN_REMOTE_HOST", "MLRUN_EXEC_CONFIG"]:
    if var in os.environ: 
        logger.info(f"Removing env var: {var}")
        del os.environ[var]

# Defer mlrun import until after env vars
# import mlrun # Moved down

logger.info(f"Pre-MLRun Import: Checking if DB file exists: {DB_FILE_PATH} - {os.path.exists(DB_FILE_PATH)}")

# --- Attempt to create the DB file MANUALLY with sqlite3 to check permissions/path ---
logger.info(f"Attempting to manually create/touch SQLite DB at: {DB_FILE_PATH} using sqlite3 library")
try:
    conn = sqlite3.connect(DB_FILE_PATH)
    conn.execute("CREATE TABLE IF NOT EXISTS test_table (id INTEGER PRIMARY KEY, name TEXT);")
    conn.commit()
    conn.close()
    if os.path.exists(DB_FILE_PATH):
        logger.info(f"SUCCESS: Manually created/opened SQLite DB file at {DB_FILE_PATH}")
        # Optionally, leave it or delete it to see if MLRun can work with an existing empty one or create its own
        # os.remove(DB_FILE_PATH) 
        # logger.info(f"Manually created DB file removed to let MLRun create it.")
    else:
        logger.error(f"FAILURE: sqlite3.connect did NOT create the file at {DB_FILE_PATH}")
except Exception as e:
    logger.error(f"ERROR manually creating SQLite DB with sqlite3: {e}", exc_info=True)
    logger.error("This suggests a fundamental issue with SQLite on your system or path/permissions.")
    raise # Stop if this basic SQLite operation fails

# --- Now import MLRun ---
import mlrun
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should be e.g., 1.7.0

logger.info(f"VERIFY (after import): mlrun.mlconf.dbpath = {mlrun.mlconf.dbpath}")
if mlrun.mlconf.dbpath != DB_CONNECTION_STRING:
    logger.error("CRITICAL MISMATCH! mlrun.mlconf.dbpath not picked up from env var.")
    # Force it, though this is a symptom of a problem
    mlrun.mlconf.dbpath = DB_CONNECTION_STRING
    logger.warning(f"Forced mlrun.mlconf.dbpath to: {mlrun.mlconf.dbpath}")
else:
    logger.info("SUCCESS: mlrun.mlconf.dbpath matches env var.")

db_type_before_project = "Unknown"
try:
    db = mlrun.get_run_db(force_reconnect=True) # force_reconnect to ensure it uses current mlconf
    db_type_before_project = str(type(db))
    logger.info(f"mlrun.get_run_db() [before project] returned: {db_type_before_project}")
    if "HTTPRunDB" in db_type_before_project:
        logger.error("DB is HTTPRunDB before project, this is wrong!")
    elif "SQLDB" in db_type_before_project or "FileDB" in db_type_before_project:
        logger.info("DB type is SQLDB/FileDB before project (Correct).")
except Exception as e:
    logger.error(f"Error from mlrun.get_run_db() [before project]: {e}", exc_info=True)
    raise

# --- 2. Project ---
logger.info(f"State of DB file '{DB_FILE_PATH}' before get_or_create_project: Exists = {os.path.exists(DB_FILE_PATH)}")

project = mlrun.get_or_create_project(name=PROJECT_NAME, context="./", user_project=False)
logger.info(f"Project object '{project.name}' obtained. Attempting project.save()...")

# Check DB file existence right before save
logger.info(f"State of DB file '{DB_FILE_PATH}' right before project.save(): Exists = {os.path.exists(DB_FILE_PATH)}")
try:
    project.save()
    logger.info("project.save() executed.")
except Exception as e:
    logger.error(f"Error during project.save(): {e}", exc_info=True)
    # We will check file existence after this anyway

if not os.path.exists(DB_FILE_PATH):
    logger.error(f"FAILURE: DB file '{DB_FILE_PATH}' NOT FOUND after project.save().")
    logger.error(f"  mlrun.mlconf.dbpath was: {mlrun.mlconf.dbpath}")
    logger.error(f"  DB type MLRun resolved to (before project) was: {db_type_before_project}")
    # Try to get the DB again to see if MLRun can connect or what error it gives
    try:
        logger.info("Attempting mlrun.get_run_db() again AFTER failed project.save()...")
        db_after_fail = mlrun.get_run_db(force_reconnect=True)
        logger.info(f"  mlrun.get_run_db() [after failed save] returned: {type(db_after_fail)}")
    except Exception as e_db_after:
        logger.error(f"  Error from mlrun.get_run_db() [after failed save]: {e_db_after}", exc_info=True)
    raise RuntimeError("DB file not created/found after project.save()")

logger.info(f"SUCCESS (tentative): DB file '{DB_FILE_PATH}' EXISTS after project.save(). Checking its validity...")

# --- Minimal check of the created DB if it exists ---
try:
    conn_check = sqlite3.connect(DB_FILE_PATH)
    cursor = conn_check.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='projects';")
    if cursor.fetchone():
        logger.info("VALIDATION SUCCESS: 'projects' table found in the MLRun-created DB.")
    else:
        logger.warning("VALIDATION WARNING: 'projects' table NOT found in the DB. It might be empty or not fully initialized by MLRun.")
    conn_check.close()
except Exception as e_val:
    logger.error(f"Error validating MLRun-created DB file: {e_val}", exc_info=True)


# --- Function definition and run (only if DB creation was successful) ---
logger.info("Proceeding to function definition and run...")
func_code = "def handler(context): context.log_result('test_stable_dbg', 'ok_stable_dbg')"
with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)

fn = project.set_function(FUNCTION_FILE_NAME, name="stable-dbg-job", kind="job", image=DOCKER_IMAGE, handler="handler")

logger.info("Attempting to run function with local=False...")
try:
    run_obj = project.run_function(fn, local=False, watch=True)
    logger.info(f"Run state: {run_obj.status.state if run_obj else 'N/A'}")
    if run_obj and run_obj.status.state == "error":
         logger.error(f"Run failed: {run_obj.status.error}")
except Exception as e:
    logger.error(f"Error during run_function: {e}", exc_info=True)

logger.info("--- Stable Version DB Test End ---")

In [ ]:
import os
import logging
import shutil

# --- 0. Setup ---
LOG_FORMAT = '%(asctime)s %(levelname)s [%(name)s] %(filename)s:%(lineno)d - %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger("StableVersionTest")

NOTEBOOK_DIR = os.path.abspath(".")
ARTIFACT_PATH = os.path.join(NOTEBOOK_DIR, "stable_artifacts")
DB_FILE_PATH = os.path.join(NOTEBOOK_DIR, "stable_mlrun.db")
DB_CONNECTION_STRING = f"sqlite:///{DB_FILE_PATH}"
PROJECT_NAME = "stable-local-project"
FUNCTION_FILE_NAME = "stable_func.py"
DOCKER_IMAGE = "mlrun/mlrun" # Or a version-compatible image if needed

# Clean slate
if os.path.exists(DB_FILE_PATH): os.remove(DB_FILE_PATH)
if os.path.exists(os.path.join(NOTEBOOK_DIR, "project.yaml")): os.remove(os.path.join(NOTEBOOK_DIR, "project.yaml"))
if os.path.exists(ARTIFACT_PATH): shutil.rmtree(ARTIFACT_PATH)
os.makedirs(ARTIFACT_PATH, exist_ok=True)

# --- 1. Set Env Vars THEN Import MLRun ---
logger.info(f"Setting OS Env MLRUN_DBPATH = {DB_CONNECTION_STRING}")
os.environ["MLRUN_DBPATH"] = DB_CONNECTION_STRING
os.environ["MLRUN_ARTIFACT_PATH"] = ARTIFACT_PATH
for var in ["MLRUN_HTTPDB__URL", "MLRUN_API_PROXY__URL", "MLRUN_REMOTE_HOST", "MLRUN_EXEC_CONFIG"]:
    if var in os.environ: del os.environ[var]

import mlrun
logger.info(f"MLRun Imported. Version: {mlrun.get_version()}") # Should now be e.g., 1.7.0

logger.info(f"VERIFY: mlrun.mlconf.dbpath = {mlrun.mlconf.dbpath}")
if mlrun.mlconf.dbpath != DB_CONNECTION_STRING:
    logger.error("CRITICAL MISMATCH with stable version!")
else:
    logger.info("SUCCESS: mlrun.mlconf.dbpath matches env var with stable version.")

# Check the DB type MLRun resolved to
try:
    db = mlrun.get_run_db()
    logger.info(f"mlrun.get_run_db() returned: {type(db)}")
    if "HTTPRunDB" in str(type(db)):
        logger.error("STABLE VERSION STILL RESOLVED TO HTTPRunDB! This is unexpected.")
    elif "SQLDB" in str(type(db)) or "FileDB" in str(type(db)): # FileDB for older versions
        logger.info("SUCCESS: DB type is SQLDB/FileDB, correct for SQLite.")
    else:
        logger.warning(f"DB type is {type(db)}, was expecting SQLDB/FileDB.")
except Exception as e:
    logger.error(f"Error getting DB with stable version: {e}", exc_info=True)
    raise

# --- 2. Project and Function ---
project = mlrun.get_or_create_project(name=PROJECT_NAME, context="./", user_project=False)
project.save()
if not os.path.exists(DB_FILE_PATH):
    logger.error("DB file NOT created by project.save() with stable version.")
    raise RuntimeError("DB file not created with stable version")
logger.info("DB file created by project.save() with stable version.")

func_code = "def handler(context): context.log_result('test_stable', 'ok_stable')"
with open(FUNCTION_FILE_NAME, "w") as f: f.write(func_code)

fn = project.set_function(FUNCTION_FILE_NAME, name="stable-job", kind="job", image=DOCKER_IMAGE, handler="handler")

logger.info("Attempting to run function with local=False with stable version...")
try:
    run_obj = project.run_function(fn, local=False, watch=True)
    logger.info(f"Run state (stable): {run_obj.status.state if run_obj else 'N/A'}")
    if run_obj and run_obj.status.state == "error":
         logger.error(f"Run failed (stable): {run_obj.status.error}")
except Exception as e:
    logger.error(f"Error during run_function (stable): {e}", exc_info=True)

logger.info("--- Stable Version Test End ---")